In [1]:
import tarfile
import os
import pathlib
import numpy as np
from time import time
# temp
import sys
from tqdm.notebook import tqdm
sys.path.append('/workspace/shared_workspace/deep-learning-models/models/vision/detection')

import tensorflow_addons as tfa
import tensorflow as tf
import horovod.tensorflow as hvd

from awsdet.models.detectors.mask_rcnn import MaskRCNN
from awsdet.datasets import DATASETS, build_dataloader
from awsdet.datasets import build_dataset, build_dataloader
from awsdet.models import build_detector
from awsdet.utils.schedulers import schedulers
from awsdet.core import CocoDistEvalmAPHook, CocoDistEvalRecallHook
from awsdet.utils.runner.hooks.logger import tensorboard, text
from awsdet.utils.runner.hooks import checkpoint, iter_timer, visualizer
from awsdet.apis.train import parse_losses, batch_processor, build_optimizer, get_root_logger
from awsdet.utils.misc import Config
import horovod.tensorflow as hvd
from awsdet.utils.runner import sagemaker_runner
from awsdet.utils.schedulers.schedulers import WarmupScheduler
import argparse

from awsdet.utils import visualize
from pycocotools.cocoeval import COCOeval

from awsdet.core.bbox import transforms
import matplotlib.pyplot as plt

from awsdet.utils.fileio import load, dump
import os.path as osp
from awsdet.core.evaluation.coco_utils import fast_eval_recall, results2json
from awsdet.core.mask.transforms import mask2result
from awsdet.core.evaluation import coco_utils
##########################################################################################
# Setup horovod and tensorflow environment
##########################################################################################

fp16 = True
hvd.init()
tf.config.experimental_run_functions_eagerly(True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": fp16})
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')


/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
cfg = Config.fromfile("/workspace/shared_workspace/deep-learning-models/models/vision/detection/configs/docker_mask_rcnn.py")

In [3]:
hasattr(cfg.data.val, 'mask')

True

In [4]:
cfg.batch_size_per_device = 1
cfg.workers_per_gpu = 1
cfg.global_batch_size = cfg.batch_size_per_device * hvd.size()
datasets = build_dataset(cfg.data.train)
tf_datasets = [build_dataloader(datasets,
                     cfg.batch_size_per_device,
                     cfg.workers_per_gpu,
                     num_gpus=hvd.size(),
                     dist=True)]

loading annotations into memory...
Done (t=15.91s)
creating index...
index created!


In [5]:
val_dataset = build_dataset(cfg.data.val)
val_tdf, val_size = build_dataloader(val_dataset,
                                     cfg.batch_size_per_device,
                                     cfg.workers_per_gpu,
                                     num_gpus=hvd.size(),
                                     dist=True,
                                     shuffle=False)

loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


In [6]:
model = build_detector(cfg.model,
                           train_cfg=cfg.train_cfg,
                           test_cfg=cfg.test_cfg)

In [7]:
_ = model(next(iter(tf_datasets[0][0])), training=True)

Starting new loop for GPU: 0


In [8]:
model.load_weights('/workspace/shared_workspace/output/011/mask_rcnn')

In [9]:
cfg.base_learning_rate = 0.01
cfg.warmup_init_lr_scale = 3.0
cfg.warmup_steps = 500
base_learning_rate = cfg.base_learning_rate
scaled_learning_rate = base_learning_rate * cfg.global_batch_size / 8
steps_per_epoch = 100
scheduler = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
            [steps_per_epoch * 8, steps_per_epoch * 10],
            [scaled_learning_rate, scaled_learning_rate*0.1, scaled_learning_rate*0.01])
warmup_init_lr = 1.0 / cfg.warmup_init_lr_scale * scaled_learning_rate
scheduler = WarmupScheduler(scheduler, warmup_init_lr, cfg.warmup_steps)
optimizer = tf.keras.optimizers.SGD(scheduler, momentum=0.9, nesterov=False)
optimizer = tf.train.experimental.enable_mixed_precision_graph_rewrite(optimizer, loss_scale='dynamic')

In [10]:
runner = sagemaker_runner.Runner(model, batch_processor, name='mask_test', 
                                     optimizer=optimizer, work_dir='/workspace/shared_workspace/logs',
                                     logger=get_root_logger(cfg.log_level), amp_enabled=True,
                                     loss_weights=cfg.loss_weights, with_mask=True)
runner.register_hook(iter_timer.IterTimerHook())
runner.register_hook(text.TextLoggerHook())
runner.register_hook(CocoDistEvalmAPHook(cfg.data.val, interval=1))

loading annotations into memory...
Done (t=2.54s)
creating index...
index created!


In [11]:
#runner.run(tf_datasets, cfg.workflow, 5)

In [12]:
steps = 1
results = []
for i, batch in tqdm(enumerate(val_tdf.take(steps)), total=steps):
    detection = runner.model(batch, training=False)
    '''bboxes = transforms.bbox_mapping_back(detection['bboxes'], batch[1][0])
    box_ints = tf.round(bboxes)
    labels = detection['labels']
    scores = detection['scores']
    result = transforms.bbox2result(bboxes, labels, 
                                          scores, num_classes=val_dataset.CLASSES+1)
    masks = mask2result(bboxes, detection['masks'], labels, batch[1][0])
    results.append((result, masks))'''

Starting new loop for GPU: 0



In [19]:
tf.gather(detection['masks'], tf.transpose(detection['labels']), axis=-1, batch_dims=0)

<tf.Tensor: shape=(100, 28, 28, 100), dtype=float32, numpy=
array([[[[2.65816669e-03, 9.94220842e-03, 2.65816669e-03, ...,
          8.23902711e-03, 9.94220842e-03, 8.32659937e-03],
         [2.53703143e-03, 9.63433925e-03, 2.53703143e-03, ...,
          8.27074144e-03, 9.63433925e-03, 8.07378255e-03],
         [5.46387723e-03, 1.42608108e-02, 5.46387723e-03, ...,
          1.29624102e-02, 1.42608108e-02, 1.14680557e-02],
         ...,
         [3.91144678e-02, 4.91722301e-02, 3.91144678e-02, ...,
          5.24398610e-02, 4.91722301e-02, 2.97808703e-02],
         [5.01184864e-03, 1.42835984e-02, 5.01184864e-03, ...,
          1.11747533e-02, 1.42835984e-02, 1.17706750e-02],
         [5.71429962e-03, 1.53320683e-02, 5.71429962e-03, ...,
          1.37409111e-02, 1.53320683e-02, 1.27117764e-02]],

        [[2.15252885e-03, 8.90429504e-03, 2.15252885e-03, ...,
          7.29603646e-03, 8.90429504e-03, 8.23565386e-03],
         [2.07466260e-03, 8.76230747e-03, 2.07466260e-03, ...,
       

In [42]:
from importlib import reload
coco_utils = reload(coco_utils)

In [43]:
result_files = coco_utils.results2json(val_dataset, results, 'temp_eval')

In [44]:
result_files

{'bbox': 'temp_eval.bbox.json', 'segm': 'temp_eval.segm.json'}

In [16]:
for detection in detections:
    bboxes = tf.round(transforms.bbox_mapping_back(detections['bboxes'], batch[1][0]))
    #bboxes_round = tf.round(transforms.bbox_mapping_back(detections['bboxes'], batch[1][0]))

TypeError: list indices must be integers or slices, not str

In [46]:
runner.model.